In [7]:
import cv2
import numpy as np

filepath_left = "img/img1.jpg"
filepath_right = "img/img2.jpg"

In [8]:
image1 = cv2.imread(filepath_left)
image2 = cv2.imread(filepath_right)

# 统一图片尺寸
height, width, _ = image1.shape
# image2 = cv2.resize(image2, (width, height))

# 转换图像为灰度图
gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

In [9]:
# 创建SIFT检测器
sift = cv2.SIFT_create()

# 寻找关键点和SIFT描述子
keypoints1, descriptors1 = sift.detectAndCompute(gray1, None)
keypoints2, descriptors2 = sift.detectAndCompute(gray2, None)

# 使用Brute Force匹配器进行特征匹配
bf = cv2.BFMatcher()
matches = bf.knnMatch(descriptors1, descriptors2, k=2)

# 进行比率测试以剔除不好的匹配
good_matches = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good_matches.append(m)

# 如果匹配点数不足，拼接可能会失败
if len(good_matches) > 5:
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    # 获取两张图之间的特征点匹配
    matches_image = cv2.drawMatches(image1, keypoints1, image2, keypoints2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    # 保存匹配结果
    cv2.imwrite('img/match.jpg', matches_image)

    # 使用RANSAC算法估计透视变换矩阵
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    # 获取第一个图像的高度和宽度
    h1, w1 = gray1.shape

    # 将第一个图像映射到第二个图像上
    result = cv2.warpPerspective(image1, M, (w1 + image2.shape[0], h1 + image2.shape[1]))

    # 将第二个图像添加到结果图像中
    result[0:image2.shape[0], 0:image2.shape[1]] = image2

    # 保存拼接结果
    cv2.imwrite('img/Panorama.jpg', result)

else:
    print("No enough good matches to create a panorama.")